In [1]:
import pandas as pd
import bs4
from datetime import datetime, timedelta
from ics import Calendar, Event

def get_slots(path_to_csv):
    
    df = pd.read_csv(path_to_csv)
    t = df[df.columns[0]].notnull()
    for i in range(1,len(df.columns)):
        t = t | df[df.columns[i]].notnull()

    df = df[t]
    df[df["Unnamed: 0"].apply(lambda x: len(str(x)) == 3)]
    df = df[df["Unnamed: 0"].apply(lambda x: len(str(x)) == 3)]

    time = df.values[0]

    d = {}
    for row in df.values:
        for i in range(len(row)):
            c = row[i]
            if len(str(c)) == 1:
                day = row[0]
                t = time[i]
                if c not in d:
                    d[c] = []
                start,end = time[i].split("-")
                start,end = start.strip(), end.strip()
                d[c].append({"day":day,"start":start, "end":end})
    
    return d

def get_course_details(course_list_csv):
    course_details = {}
    df = pd.read_csv("List of Courses.csv",header=1)
    
    for code, name, slot in zip(df["Course Code"],df["Title"],df["Slot"]):
        course_details[code] = {"title":name,"slot":slot} 
    
    return course_details

def get_registered_courses(registration_sheet_path):
    with open(registration_sheet_path, 'r') as content_file:
        content = content_file.read()

    soup = bs4.BeautifulSoup(content)
    tables = soup.find_all("table")
    df = pd.read_html(str(tables[3]))[0]

    courses = df["CourseCode"].dropna()
    return [i for i in courses]

def generate_event_details(courses, slot_details, event_details):
    code, title, day,start,end = [],[],[],[],[]
    time = []
    d = []
    for cc in courses:
        for slot in slot_details[event_details[cc]["slot"]]:
            t = slot.copy()
            t["code"] = cc
            t["title"] = event_details[cc]["title"]
            d.append(t)
          

    df = pd.DataFrame(d)
    return df

def generate_ical_file(event_details,output="output.ics"):

    end = datetime(2020, 5, 6, 23, 59, 59, 0)

    date = datetime.today()
    c = Calendar()
    while date < end: 
        day = date.strftime('%a')
        for _, row in event_details[event_details["day"]==day].iterrows():
            e = Event()
            e.name = row["title"]
            e.begin = date.strftime("%Y-%m-%d ")+ row["start"] + "+05:30"
            e.end =  date.strftime("%Y-%m-%d ")+ row["end"] + "+05:30"
            c.events.add(e)
        date += timedelta(days=1)

    with open(output, 'w') as f:
        f.write(str(c))
    
    return c

def get_slot_details_with_rooms(Room_table_csv):
    df = pd.read_csv(Room_table_csv,header=1)

    df["Day"].fillna(method="ffill", inplace=True)

    df["Day"] = df["Day"].apply(lambda x: x[:3])

    d  = {}
    for row in df.values:
        for i in range(len(row)):
            c = row[i]    
            if ":" in str(c):
                Type,code = c.split(":")
                day = row[0]
                time = df.columns[i]
                start,end = time.split("-")
                start,end = start.strip(),end.strip()
                Type = Type.strip()
                room = row[1]
                if code not in d:
                    d[code] = []
                d[code].append({"day":day,"room":room, "start":start,"end":end ,"type":Type})
    return d

In [ ]:
def main(course_registration_html, output="output.ics"):
    slot_details = get_slots("Slot System.csv")

    course_details = get_course_details("List of Courses.csv")

    registered_courses = get_registered_courses(course_registration_html)

    event_details = generate_event_details(registered_courses, slot_details, course_details)

    c = generate_ical_file(event_details,output)
    
    return c

In [ ]:
def main_new(course_registration_html, output="output.ics"):
    registered_courses = get_registered_courses(course_registration_html)
    d = get_slot_details_with_rooms(Room_table_csv="Room Wise Time Table.csv")
    course_details = get_course_details("List of Courses.csv")
    registerd_d = []
    for course in registered_courses:
        for slot in d[course]:
            t = slot.copy()
            t["title"] = course_details[course]["title"]
            t["code"] = course
            registerd_d.append(t)
    
    event_details = pd.DataFrame(registerd_d)
    c = generate_ical_file(event_details,output)
    
    return c

In [ ]:
main_new("Course Registration.html",output="arjun_new.ics")

In [ ]:
slot_details = get_slots("Slot System.csv")
course_details = get_course_details("List of Courses.csv")

In [ ]:
main("Course Registration.html",output="arjun.ics")

# Indraneel

In [ ]:
c  = main("Course Registration Indraneel.html",output="indraneel.ics")

In [ ]:
c.event

In [ ]:
course_registration_html = "Course Registration Indraneel.html"


In [ ]:
registered_courses = get_registered_courses(course_registration_html)

In [ ]:
registered_courses

In [ ]:
del registered_courses[2]  # Removing Reliability

In [ ]:
registered_courses.append("CS 419/619") #adding computer vision

In [ ]:
registered_courses

In [ ]:
del registered_courses[3] # removed 2d 

In [ ]:
registered_courses.append("IHS 443/643") #adding contemporary

In [ ]:
event_details = generate_event_details(registered_courses, slot_details, course_details)

In [ ]:
event_details

In [ ]:
event_details.pivot_table(index="day", columns="start", values="title", aggfunc=lambda x: x)

In [ ]:
pc = []
for cc in course_details:
    if course_details[cc]["slot"] in slot_details:
        flag = True
        for day,_ in slot_details[course_details[cc]["slot"]]:
            if day == "Wed" or day=="Tue":
                flag = False
                break
        if flag:
            if cc.split()[1][0] in ["4","6"]:
                pc.append(cc)

In [ ]:
for cc in pc:
    if cc not in registered_courses:
        print(course_details[cc]["title"], cc)

In [ ]:
course_details['HS 641'][]

In [ ]:
event_details.pivot_table(index="day", columns="start", values="title", aggfunc=lambda x: x)

# Arjun

In [39]:
course_registration_html = "Course Registration.html"
registered_courses = get_registered_courses(course_registration_html)



del registered_courses[2] # Removed Contemporary Indian Thought

registered_courses.append("CS 420/620") # Added Embedded 

del registered_courses[3]

registered_courses.append("IHS 443/643") # added Contemporary short fiction

registered_courses.append("CS 417/617") # what if Auditing Crypto

registered_courses.append("EE 446/646") # what if Auditing Imformation Coding theory

registered_courses.append("HS 412/612") # what if Auditing  contemporary  Indian thought

registered_courses.append("ME 472/672") # what if Auditing Reliablity 

registered_courses.append("IHS 444") # what if Auditing Literature of 20th century

registered_courses.append("HS 418/618") # what if Auditing Sustainibility Studies 

registered_courses.append("MM 402/602") # what if Auditing Design and Selection of materials

In [40]:
registered_courses.append("CE 404") # what if Auditing Galactic and Extragalactic Astronomy

In [41]:
event_details = generate_event_details(registered_courses, slot_details, course_details)
all([i==1 for i in event_details.groupby(['day','start']).size()]) # Check valid

True

In [42]:
len(registered_courses)

14

In [43]:
event_details = generate_event_details(registered_courses, slot_details, course_details)
event_details

,day,start,end,code,title
0,Mon,09:00,09:55,CE 438,Probabilistic and Statistical Methods in Civil...
1,Tue,11:00,11:55,CE 438,Probabilistic and Statistical Methods in Civil...
2,Fri,09:00,09:55,CE 438,Probabilistic and Statistical Methods in Civil...
3,Mon,13:00,13:55,CS 419/619,Computer Vision
4,Thu,15:00,15:55,CS 419/619,Computer Vision
5,Fri,16:00,16:55,CS 419/619,Computer Vision
6,Mon,14:00,14:55,IHS 402,Twentieth Century World History: Critical Pers...
7,Tue,13:00,13:55,IHS 402,Twentieth Century World History: Critical Pers...
8,Thu,13:00,13:55,IHS 402,Twentieth Century World History: Critical Pers...
9,Tue,15:00,15:55,MM 430/730,Two Dimensional Materials and Electronic Devices


In [44]:
event_details = generate_event_details(registered_courses, slot_details, course_details)
event_details.pivot_table(index="day", columns="start", values="title", aggfunc=lambda x: x)

start,09:00,10:00,11:00,13:00,14:00,15:00,16:00,17:00
day,,,,,,,,
Fri,Probabilistic and Statistical Methods in Civil...,Design and Selection of Materials,Sustainability Studies,Cryptography and Network Security,Design of Structures-III,Literature of the Twentieth Century,Computer Vision,Contemporary Short Fiction
Mon,Probabilistic and Statistical Methods in Civil...,Design and Selection of Materials,Contemporary Short Fiction,Computer Vision,Twentieth Century World History: Critical Pers...,Cryptography and Network Security,Embedded Systems,Literature of the Twentieth Century
Sat,NaN,NaN,NaN,NaN,Contemporary Indian Thought,Contemporary Indian Thought,NaN,NaN
Thu,Reliability Engineering,Design and Selection of Materials,Contemporary Short Fiction,Twentieth Century World History: Critical Pers...,Sustainability Studies,Computer Vision,Two Dimensional Materials and Electronic Devices,Information and Coding Theory
Tue,Reliability Engineering,Information and Coding Theory,Probabilistic and Statistical Methods in Civil...,Twentieth Century World History: Critical Pers...,Design of Structures-III,Two Dimensional Materials and Electronic Devices,Cryptography and Network Security,Embedded Systems
Wed,Reliability Engineering,Information and Coding Theory,Sustainability Studies,Design of Structures-III,Literature of the Twentieth Century,Two Dimensional Materials and Electronic Devices,Embedded Systems,Contemporary Indian Thought


In [ ]:
registered_courses

In [ ]:
schedule = event_details.pivot_table(index="day", columns="start", values="title", aggfunc=lambda x: x)

# Prathamesh

In [ ]:
course_registration_html = "Course Registration Prathamesh.html"
registered_courses = get_registered_courses(course_registration_html)

In [ ]:
registered_courses

In [ ]:
registered_courses.append("CS 417/617") # crypto
registered_courses.append("CS 420/620") # embedded
registered_courses.append("CE 438") # probability


In [ ]:
event_details = generate_event_details(registered_courses, slot_details, course_details)
event_details.pivot_table(index="day", columns="start", values="title", aggfunc=lambda x: x)

In [ ]:
schedule

# Exporting json

In [25]:
slot_details = get_slots("Slot System.csv")
course_details = get_course_details("List of Courses.csv")

for course in course_details:
    if course_details[course]["slot"] in slot_details:
        course_details[course]["slots"] =  slot_details[course_details[course]["slot"]]
    else:
        course_details[course]["slots"] = []

In [26]:
rooms = get_slot_details_with_rooms("Room Wise Time Table.csv")

In [27]:
course_details['MA 106(A)']

{'title': 'Linear Algebra and Ordinary Differential Equation-I', 'slot': 'A'}

In [28]:
rooms['MA 106(A)']

[{'day': 'Mon',
  'room': '1E -104',
  'start': '10:00',
  'end': '10:55',
  'type': 'L'},
 {'day': 'Thu',
  'room': '1E -104',
  'start': '10:00',
  'end': '10:55',
  'type': 'L'},
 {'day': 'Fri',
  'room': '1E -104',
  'start': '10:00',
  'end': '10:55',
  'type': 'L'}]

In [29]:
rooms_not_found = {}

for course_code in course_details:
    slot_code = course_details[course_code]["slot"]
    if slot_code in slot_details:
        course_details[course_code]["slots"] = []
        for slot1 in slot_details[slot_code]:
            flag = True
            if course_code in rooms:
                for slot2 in rooms[course_code]:
                     if slot1["day"]==slot2["day"] and slot1["start"]==slot2["start"] and slot1["end"] == slot2["end"]:
                        course_details[course_code]["slots"].append(deepcopy(slot2))
                        flag = False
                        break
            if flag:
                temp = deepcopy(slot1)
                temp["room"] = "room not found"
                temp["type"] = "unknown"
                course_details[course_code]["slots"].append(temp)
                if course_code not in rooms_not_found:
                    rooms_not_found[course_code] = []
                rooms_not_found[course_code].append(deepcopy(slot1))
                
    else:
        course_details[course_code]["slots"] = []
        
            
                                
                
#         if course_code in rooms:
#             for slot1 in course_details[course_code]["slots"]:
#                     flag = True
#                     for slot2 in rooms[course_code]:
#                          if slot1["day"]==slot2["day"] and slot1["start"]==slot2["start"] and slot1["end"] == slot2["end"]:
                                

#                         if slot1["day"]==slot2["day"] and slot1["start"]==slot2["start"] and slot1["end"] == slot2["end"]:
#                             slot1["room"] = slot2["room"]
#                             slot1["type"] = slot2["type"]

#                             flag=False
#                             break

#                         if flag:
#                             slot1["room"] = "room not found"

#                             if course_code not in rooms_not_found:
#                                 rooms_not_found[course_code] = []
#                             rooms_not_found[course_code].append(slot1)            
#     else:
#         rooms_not_found[course_code] = []

In [30]:
course_details

{'HS 641': {'title': 'English Communication Skills', 'slot': '-', 'slots': []},
 'MA 106(A)': {'title': 'Linear Algebra and Ordinary Differential Equation-I',
  'slot': 'A',
  'slots': [{'day': 'Mon',
    'room': '1E -104',
    'start': '10:00',
    'end': '10:55',
    'type': 'L'},
   {'day': 'Thu',
    'room': '1E -104',
    'start': '10:00',
    'end': '10:55',
    'type': 'L'},
   {'day': 'Fri',
    'room': '1E -104',
    'start': '10:00',
    'end': '10:55',
    'type': 'L'}]},
 'CH 103': {'title': 'Chemistry',
  'slot': 'A',
  'slots': [{'day': 'Mon',
    'room': '1B -201',
    'start': '10:00',
    'end': '10:55',
    'type': 'L'},
   {'day': 'Thu',
    'room': '1B -201',
    'start': '10:00',
    'end': '10:55',
    'type': 'L'},
   {'day': 'Fri',
    'room': '1B -201',
    'start': '10:00',
    'end': '10:55',
    'type': 'T'}]},
 'CS 202': {'title': 'Automata Theory and Logic',
  'slot': 'A',
  'slots': [{'day': 'Mon',
    'room': '1E -505',
    'start': '10:00',
    'end': '

In [19]:
import json

In [36]:
with open('data_unindented.json', 'w') as fp:
    json.dump(course_details, fp,sort_keys=True)

In [31]:
rooms_not_found

{'BSE 102': [{'day': 'Fri', 'start': '09:00', 'end': '09:55'}],
 'IC 153': [{'day': 'Tue', 'start': '10:00', 'end': '10:55'},
  {'day': 'Thu', 'start': '17:00', 'end': '17:55'}],
 'EE 104': [{'day': 'Thu', 'start': '14:00', 'end': '14:55'}],
 'CS 103': [{'day': 'Thu', 'start': '14:00', 'end': '14:55'}],
 'PH 105': [{'day': 'Fri', 'start': '12:00', 'end': '12:55'}],
 'ME 106': [{'day': 'Fri', 'start': '12:00', 'end': '12:55'}],
 'ME 472/672': [{'day': 'Thu', 'start': '09:00', 'end': '09:55'}],
 'ES 302': [{'day': 'Mon', 'start': '17:00', 'end': '17:55'},
  {'day': 'Tue', 'start': '17:00', 'end': '17:55'},
  {'day': 'Sat', 'start': '11:00', 'end': '11:55'}],
 'HS 302': [{'day': 'Mon', 'start': '17:00', 'end': '17:55'},
  {'day': 'Tue', 'start': '17:00', 'end': '17:55'},
  {'day': 'Sat', 'start': '11:00', 'end': '11:55'}],
 'PH 106': [{'day': 'Fri', 'start': '17:00', 'end': '17:55'}],
 'MA 106(B)': [{'day': 'Mon', 'start': '12:00', 'end': '12:55'},
  {'day': 'Thu', 'start': '12:00', 'end'

In [7]:
co = 0
rooms_not_found = {}
for c in course_details:
    if c not in rooms:
        print(course_details[course_code]["slots"])
        for slot1 in course_details[course_code]["slots"]:
            if course_code not in rooms_not_found:
                rooms_not_found[course_code] = []
            rooms_not_found[course_code].append(slot1)
        co += 1
co, len(rooms_not_found)

NameError: name 'course_code' is not defined

In [9]:
len(rooms_not_found)

192

In [ ]:
rooms_not_found

In [ ]:
rooms_not_found

In [14]:
course_details["BSE 102"]

{'title': 'Bio-Sciences',
 'slot': 'B',
 'slots': [{'day': 'Mon', 'start': '09:00', 'end': '09:55'},
  {'day': 'Tue', 'start': '11:00', 'end': '11:55'},
  {'day': 'Fri', 'start': '09:00', 'end': '09:55'}]}

In [17]:
rooms["BSE 102"]

[{'day': 'Mon',
  'room': '1E -104',
  'start': '09:00',
  'end': '09:55',
  'type': 'L'},
 {'day': 'Tue',
  'room': '1E -104',
  'start': '11:00',
  'end': '11:55',
  'type': 'L'}]

In [ ]:
course_details["BSE 102"]

In [13]:
len(course_details)

192

In [7]:
[1,2,3,4,5,6].copy()

[1, 2, 3, 4, 5, 6]

In [8]:
from copy import deepcopy

In [35]:
len(rooms_not_found)

23

In [20]:
with open('data_unindented.json', 'r') as fp:
    data = json.load(fp)

In [22]:
for c in data:
    print(f"<option value='{c}'>{c} : {data[c]['title']}</option>")
   

<option value='AA 202N'>AA 202N : Astronomical Techniques</option>
<option value='AA 472N/672N'>AA 472N/672N : Galactic and Extragalactic Astronomy</option>
<option value='AA 474N/674N'>AA 474N/674N : Radio Astronomy</option>
<option value='AA 478/678'>AA 478/678 : Space Weather</option>
<option value='AA 602N'>AA 602N : Advanced Topics in Astronomy and Astrophysics</option>
<option value='AA 606'>AA 606 : Random Signals and Applied Kalman Filtering</option>
<option value='AA 608N'>AA 608N : Astrostatistics</option>
<option value='AA 652'>AA 652 : Astronomy Laboratory-II</option>
<option value='BSE 102'>BSE 102 : Bio-Sciences</option>
<option value='BSE 404/604'>BSE 404/604 : Biomedical Imaging</option>
<option value='BSE 602'>BSE 602 : Modern NMR Spectroscopy</option>
<option value='BSE 606'>BSE 606 : Molecular Virology and Viral Pathogenesis</option>
<option value='BSE 607'>BSE 607 : Bioremediation Principles and Practices</option>
<option value='BSE 610'>BSE 610 : Cell and Developme

In [46]:
small_d = {}
for course in registered_courses:
    small_d[course] = data[course]

In [53]:
registered_courses

['CE 438',
 'CS 419/619',
 'IHS 402',
 'MM 430/730',
 'CS 420/620',
 'IHS 443/643',
 'CS 417/617',
 'EE 446/646',
 'HS 412/612',
 'ME 472/672',
 'IHS 444',
 'HS 418/618',
 'MM 402/602',
 'CE 404']

In [48]:
ans = []
for course in small_d:
    for slot in small_d[course]["slots"]:
        t = deepcopy(slot)
        t["code"] = course
        t["title"] = small_d[course]["title"]
        ans.append(t)

In [50]:
event_details = pd.DataFrame(ans)

In [59]:
event_details["descriptions"] = event_details["title"] +" " +event_details["room"] 

In [60]:
event_details.pivot_table(index="day", columns="start", values=["descriptions"], aggfunc=lambda x: x)

descriptions  \
start                                              09:00   
day                                                        
Fri    Probabilistic and Statistical Methods in Civil...   
Mon    Probabilistic and Statistical Methods in Civil...   
Sat                                                  NaN   
Thu               Reliability Engineering room not found   
Tue                      Reliability Engineering 1E -305   
Wed                      Reliability Engineering 1E -305   

                                                  \
start                                      10:00   
day                                                
Fri    Design and Selection of Materials 1E -406   
Mon    Design and Selection of Materials 1E -406   
Sat                                          NaN   
Thu    Design and Selection of Materials 1E -406   
Tue        Information and Coding Theory 1E -205   
Wed        Information and Coding Theory 1E -205   

                                                          \
start                                              11:00   
day                                                        
Fri                       Sustainability Studies 1E -304   
Mon                   Contemporary Short Fiction 1E -205   
Sat                                                  NaN   
Thu                   Contemporary Short Fiction 1E -205   
Tue    Probabilistic and Statistical Methods in Civil...   
Wed                       Sustainability Studies 1E -304   

                                                          \
start                                              13:00   
day                                                        
Fri            Cryptography and Network Security 1E -506   
Mon                              Computer Vision 1E -304   
Sat                                                  NaN   
Thu    Twentieth Century World History: Critical Pers...   
Tue    Twentieth Century World History: Critical Pers...   
Wed                     Design of Structures-III 1E -408   

                                                          \
start                                              14:00   
day                                                        
Fri                     Design of Structures-III 1E -408   
Mon    Twentieth Century World History: Critical Pers...   
Sat                  Contemporary Indian Thought 1B -201   
Thu                       Sustainability Studies 1E -304   
Tue                     Design of Structures-III 1E -408   
Wed          Literature of the Twentieth Century 1E -104   

                                                          \
start                                              15:00   
day                                                        
Fri          Literature of the Twentieth Century 1E -104   
Mon            Cryptography and Network Security 1E -506   
Sat                  Contemporary Indian Thought 1B -201   
Thu                              Computer Vision 1E -304   
Tue    Two Dimensional Materials and Electronic Devic...   
Wed    Two Dimensional Materials and Electronic Devic...   

                                                          \
start                                              16:00   
day                                                        
Fri                              Computer Vision 1E -304   
Mon                             Embedded Systems 1E -506   
Sat                                                  NaN   
Thu    Two Dimensional Materials and Electronic Devic...   
Tue            Cryptography and Network Security 1E -506   
Wed                             Embedded Systems 1E -506   

                                                    
start                                        17:00  
day                                                 
Fri             Contemporary Short Fiction 1E -205  
Mon    Literature of the Twentieth Century 1E -104  
Sat                                            NaN  
Thu        

In [56]:
course_details[registered_courses[4]]

{'title': 'Embedded Systems',
 'slot': 'X',
 'slots': [{'day': 'Mon',
   'room': '1E -506',
   'start': '16:00',
   'end': '16:55',
   'type': 'L'},
  {'day': 'Tue',
   'room': '1E -506',
   'start': '17:00',
   'end': '17:55',
   'type': 'T'},
  {'day': 'Wed',
   'room': '1E -506',
   'start': '16:00',
   'end': '16:55',
   'type': 'L'}]}